## An example workflow for schism

This is a simple example for SCHISM that assumes that the grid file is given. Since no input options for meteo or bathymetry is given they are retrieved from an erddap server (GFS, srtmplus).

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.model as pm
from pyPoseidon.utils import data,point,obs
import os
import glob
import xarray as xr
import geopandas as gp

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("dark_background")

In [ ]:
from hvplot import xarray, pandas

In [ ]:
%matplotlib widget

## Choose data path

In [ ]:
DATA_PATH = os.path.dirname(pyPoseidon.__file__)   # get path to the test data installed with the package
DATA_PATH

..or..

In [ ]:
DATA_PATH = os.getcwd() 
DATA_PATH  = os.path.abspath(os.path.join(DATA_PATH, os.pardir)) + '/pyPoseidon/'
DATA_PATH

In [ ]:
PWD = os.getcwd()
PWD

### Define case

This is a simple case with 12h time frame. It is the first step for the Hindcast_Schism.ipynb

In [ ]:
#define in a dictionary the properties of the model..
case={'solver':'schism',
     'grid_file': DATA_PATH + '/tests/data/hgrid.gr3', 
     'manning':.12,
     'windrot':0.00001,
     'tag':'schism',
     'start_date':'2017-10-1 0:0:0',
     'time_frame':'12H',
     'meteo_files' : [DATA_PATH + '/tests/data/erai.grib'], #meteo files
#     'epath':'/path_to_folder/schism.ex_VL', #folder for solver executables
     'ncores': 4 , #number of cores
     'rpath':PWD + '/test/schism/', #location of calc folder
     'conda_env': 'pyPoseidon', # optional conda env for the solver
     'update':['meteo','model'], #set which component should be updated  (meteo,dem,model)
     'monitor':True, # get time series for observation points
     'parameters':{'dt':400, 'rnday':.5, 'hotout':1, 'ihot':0,'nspool':9, 'ihfskip':36, 'hotout_write':108 } # set param.in components
    }

## Initialize

In [ ]:
case.update({'coastal_monitoring':True})#, 'station_flags':[1, 0 ,0 ,0, 0, 0, 0, 0, 0]}) # make all lan boundary points station points

In [ ]:
#initialize a model
b = pm(**case)

## Step by Step process

In [ ]:
b.create()

In [ ]:
b.output()

In [ ]:
b.save()

In [ ]:
b.set_obs()

In [ ]:
b.run()

## ... or simply in one step


In [ ]:
#initialize a model
b = pm(**case)
b.execute()

## Results

In [ ]:
b.results() # combine output

In [ ]:
ofiles = glob.glob(b.rpath + 'outputs/schout_[!0]*.nc')
ofiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
ofiles

In [ ]:
out = xr.open_mfdataset(ofiles,combine='by_coords',concat_dim='time', data_vars ='minimal')

In [ ]:
out.depth

In [ ]:
out.load()

In [ ]:
out.depth.max(), out.depth.min()

In [ ]:
f, ax = out.pplot.contourf(var='depth' ,vmin=out.depth.min(), vmax=out.depth.max())

In [ ]:
gr, ax = out.pplot.grid(lw=.3, markersize=1)

In [ ]:
# over plot the station points
b.stations.loc[:,['SCHISM_hgrid_node_x','SCHISM_hgrid_node_y']].plot.scatter(ax=ax, x='SCHISM_hgrid_node_x',y='SCHISM_hgrid_node_y',s=50)

In [ ]:
out.pplot.quiver(var='dahv',it=4,scale=.1)

### Visualize

In [ ]:
v = out.pplot.frames(var='elev',title='SSH')#, coastlines=b.coastlines)#,vmin=-.5, vmax=.5, nv=20)
v

In [ ]:
v.save('test/elev.mp4')

In [ ]:
out.pplot.qframes(var='wind_speed',title='Vel',scale=30.)

## Observation points / Validation

## Load schism station points data

In [ ]:
b.get_obs() 

In [ ]:
b.time_series

In [ ]:
# compare with map values
k = 1
b.time_series.elev.sel(point=k).hvplot() * out.elev.sel(nSCHISM_hgrid_node=b.stations.gindex.loc[k]).hvplot.scatter()

## get tide_gauge observations

In [ ]:
m = obs.obs(**b.__dict__) 

In [ ]:
m.locations

In [ ]:
# Get with index number
p = m.iloc(0)
print (p.head())
print (p.tail())

In [ ]:
#get with Station Name  
p = m.loc('Iceland - Reykjavik')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
plat,plon = m.locations.loc[0,['lat','lon']]

In [ ]:
ts = point.point(solver = 'schism', lon=plon,lat=plat,dataset=out, var='elev')
ts.time_series.head()

In [ ]:
## Join the graphs
ax = p.plot()#figsize=(10,8))
ts.time_series.plot(ax=ax,color=['r'])
ax.legend(list(p.columns.values) + ['Schism'], loc='center left',bbox_to_anchor=(1.0, 0.5))

.. or ..

In [ ]:
## Join the graphs
p.hvplot(figsize=(14,10)) * ts.time_series.hvplot.scatter(label='schism nearest points',color='red') * b.time_series.elev.sel(point=1).hvplot(color='yellow',label='schism station point')